## Metaclasses and Attributes
Metaclasses let you intercept Python's `class` statement and provide special behaviour each time a class is defined. However, dynamic attributes enable you to override objects and cause unexpected side effects.

### Use Plain Attributes Instead of Setter and Getter Methods
In Python you don't need to implement explicit setter and getter methods. Instead you should always start your implementations with simple public attributes:

In [3]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.currect = 0

r1 = Resistor(50e3)
r1.ohms = 10e3

These attributes make operations like incrementing in place natureal and clear:

In [4]:
r1.ohms += 5e3

Later, if you decide you need special behaviour when an attribute is set, you can migrate to the `@property` decorator and its corresponding `setter` attribute. Below a new subclass of `Resistor` is defined that allows the current to vary by assigning the `voltage` property. For this code to work, the names of both the setter and he getter methods must match the intended property name:

In [5]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):  # the getter
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

Now assigning the voltage property will runt he voltage setter which in turn will update the `current` attribute of the object to match:

In [6]:
r2 = VoltageResistance(1e3)
print(f"Before: {r2.current:.2f} amps")
r2.voltage = 10
print(f"After: {r2.current:.2f} amps")

Before: 0.00 amps
After: 0.01 amps


Another advantage of using a `setter` on a property is validation on values and type checking for values passed to the class. Here, we define a class that ensures that all resistance values are above zero ohms:

In [7]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f"Ohms must be > 0; got {ohms}")
        self._ohms = ohms